### Sending to Generators
Requires generator to be primed (in a suspended state) before sending data into it

In [1]:
from inspect import getgeneratorstate as gs


def echo():
    while True:
        received = yield
        print(f"Echo: {received}")

In [2]:
e = echo()
gs(e)

'GEN_CREATED'

In [3]:
try:
    e.send("test")  # generator hasn't run yet, it's not possible to send values into it
except TypeError as exc:
    print(exc)

can't send non-None value to a just-started generator


In [4]:
gs(e)

'GEN_CREATED'

In [5]:
next(e)

In [6]:
e.send("Hi there!")

Echo: Hi there!


In [7]:
gs(e)

'GEN_SUSPENDED'

In [8]:
e.send("Sending into generator!")

Echo: Sending into generator!


In [9]:
help(e.send)

Help on built-in function send:

send(...) method of builtins.generator instance
    send(arg) -> send 'arg' into generator,
    return next yielded value or raise StopIteration.



In [10]:
def squares(n):
    for i in range(n):
        yield i ** 2

In [11]:
s = squares(10)

In [12]:
next(s), next(s), next(s)

(0, 1, 4)

In [13]:
s.send(11)  # send value doesn't have to be assigned to anything

9

In [14]:
def echo():
    while True:
        received = yield
        print(f"Echo: {received}")

e = echo()

In [15]:
e.send(None)  # it's fine to send `None`!

In [16]:
gs(e)

'GEN_SUSPENDED'

In [17]:
e.send("real value")

Echo: real value


In [18]:
def squares(n):
    for i in range(n):
        received = yield i ** 2
        print("Received", received)

In [19]:
s = squares(5)

In [20]:
next(s)

0

In [21]:
s.send(11)

Received 11


1

In [22]:
s.send(12)

Received 12


4

In [23]:
s.send(None)

Received None


9

In [24]:
s.send("test")

Received test


16

In [25]:
try:
    s.send("12345")
except StopIteration as e:
    print("Stop Iteration exception!")
    print(e)

Received 12345
Stop Iteration exception!



In [26]:
def echo(max_times):
    for _ in range(max_times):
        received = yield
        print("Echo:", received)
    print("max times reached")


In [27]:
e = echo(3)

In [28]:
next(e)

In [29]:
e.send(1)

Echo: 1


In [30]:
e.send(2)

Echo: 2


In [31]:
try:
    e.send(3)
except StopIteration as exc:
    print(exc)

Echo: 3
max times reached



In [32]:
def averager():
    total = 0
    count = 0
    def inner(value):
        nonlocal total, count
        total += value
        count += 1
        return total / count
    return inner


In [33]:
def running_averages(iterable):
    avg = averager()
    for value in iterable:
        running_avg = avg(value)
        print(running_avg)


In [34]:
running_averages([1,2,3,4,5])

1.0
1.5
2.0
2.5
3.0


In [35]:
def running_averager():
    total = 0
    count = 0
    running_avg = None
    while True:
        value = yield running_avg
        count += 1
        total += value
        running_avg = total / count

In [36]:
ra = running_averager()
next(ra)

In [37]:
ra.send(10)

10.0

In [38]:
ra.send(2)

6.0

In [39]:
ra.send(33)

15.0

In [40]:
ra.send(-2)

10.75

In [41]:
def running_averages(iterable):
    averager = running_averager()
    next(averager)
    for v in iterable:
        running_average = averager.send(v)
        print(running_average)
        

In [42]:
running_averages([1,2,3,4,5,6,7,8,9])

1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0
